<img src='../figures/capa_nm_artenote.png'></a>

# Embedding - Atributos Latentes - Entradas categóricas

Este notebook apresenta o conceito de embedding e como usá-lo no PyTorch, através dos seguintes exemplos numéricos:
- Rede com entrada categórica (one-hot) e camada densa linear
- Embedding como forma eficiente de tratar entrada categórica

## Importação

In [1]:
from collections import OrderedDict
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable

## Entrada categórica

Uma variável categórica pode ter um valor dentro de um conjunto limitado que represente uma categoria nominal.
Alguns exemplos de variáveis categóricas:
- Grupo sanguíneo: A, B, AB or O.
- Cidade onde uma pessoa reside
- Cor de um produto: vermelho, verde, azul
- Uma palavra, dentro de um vocabulário limitado

# Rede neural com entrada categórica

Quando a rede neural possui entradas categóricas, temos a necessidade de colocá-lo na forma 
categórica utilizando a codificação *one-hot*. 
Iremos fazer um exemplo de rede neural com apenas uma camada densa e entrada com 
dados categóricos com os seguintes parâmetros:
- entrada categórica pertencente a um conjunto de 20 classes (n_classes)
- entrada é constituída de 10 amostras categóricos (n_amostras)
- cada amostra é um número (id) entre 0 e 19: [19, 10, 0, 1, 7, 5, 0, 1, 15, 2]
- camada densa linear com 5 neurônios (n_neuronios)

In [2]:
n_classes = 20
n_neuronios = 5
n_amostras = 10

## Diagrama da rede neural com entradas categóricas de uma camada e sem bias

<img src='../figures/Embedding_neural.png', width = 400></img>

### Criação da codificação categórica (one-hot) da sequência de entrada

In [3]:
data = np.array([19, 10, 0, 1, 7, 5, 0, 1, 15, 2])
data, data.shape

(array([19, 10,  0,  1,  7,  5,  0,  1, 15,  2]), (10,))

Codificação one-hot é feita utilizando `np.eye` indexado pela entrada:

In [4]:
data_oh = np.eye(n_classes)[data] # to categorical
print(data_oh.astype(np.int))

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


## Criação do modelo da rede densa com 5 camadas

In [5]:
linear = nn.Linear(n_classes,n_neuronios,bias=False)
linear

Linear (20 -> 5)

### Criação dos pesos da rede

Como ilustração, iremos inicializar a rede com pesos de modo que possamos identificar quando cada conjunto de pesos
for utilizado para cada símbolo categórico:
- quando a categoria for $i$, os neurônios de saída devem receber os valores $[i,2i,3i,4i,5i]$.

Os pesos da rede possuem 20 linhas (uma para cada classes de entrada) por 5 colunas (atributos de cada categoria):

In [6]:
W = np.arange(1,n_neuronios+1).reshape(-1,1).dot(np.arange(n_classes).reshape(1,-1))

my_weights = OrderedDict([
    ('weight',  torch.FloatTensor(W.astype(np.float)))])

linear.load_state_dict(my_weights) # inicializa pesos da camada linear
linear.state_dict()

OrderedDict([('weight', 
              
              Columns 0 to 12 
                  0     1     2     3     4     5     6     7     8     9    10    11    12
                  0     2     4     6     8    10    12    14    16    18    20    22    24
                  0     3     6     9    12    15    18    21    24    27    30    33    36
                  0     4     8    12    16    20    24    28    32    36    40    44    48
                  0     5    10    15    20    25    30    35    40    45    50    55    60
              
              Columns 13 to 19 
                 13    14    15    16    17    18    19
                 26    28    30    32    34    36    38
                 39    42    45    48    51    54    57
                 52    56    60    64    68    72    76
                 65    70    75    80    85    90    95
              [torch.FloatTensor of size 5x20])])

In [9]:
data_var = Variable(torch.Tensor(data_oh))
data_var

Variable containing:

Columns 0 to 12 
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     1     0     0
    1     0     0     0     0     0     0     0     0     0     0     0     0
    0     1     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     1     0     0     0     0     0
    0     0     0     0     0     1     0     0     0     0     0     0     0
    1     0     0     0     0     0     0     0     0     0     0     0     0
    0     1     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     1     0     0     0     0     0     0     0     0     0     0

Columns 13 to 19 
    0     0     0     0     0     0     1
    0     0     0     0     0     0     0
    0     0     0     0     0     0     0
    0     0     0     0     0     0 

## Predição com as 10 amostras: [19, 10, 0, 1, 7, 5, 0, 1, 15, 2]

<img src = '../figures/Embedding_categorical_predict.png', width=800></img>

Observe que a predição da rede com a sequência categórica acima é feita com a substituição
da categoria com os 5 atributos de cada classe.

In [10]:
pp = linear(data_var)
pp

Variable containing:
   19    38    57    76    95
   10    20    30    40    50
    0     0     0     0     0
    1     2     3     4     5
    7    14    21    28    35
    5    10    15    20    25
    0     0     0     0     0
    1     2     3     4     5
   15    30    45    60    75
    2     4     6     8    10
[torch.FloatTensor of size 10x5]

# Embedding como implementação eficiente de entradas categóricas

Nesta implementação de rede neural com entrada categórica, existem dois fatores que dificultam a sua
implementação eficiente:
- necessidade de se fazer a codificação para categórico antes de colocá-lo na rede
- se o número de classes for muito alto, a implementação pode se tornar muito ineficiente. É comum
  termos centenas de milhares de classes, como é o caso de palavras dentro de um vocabulário.
  
A camada `Embedding` resolve estes dois problemas de forma eficiente:
- faz a codificação categórica automaticamente e já retorna a aplicação dos pesos nos valores categóricos

Assim, a camada `Embedding` é sempre uma camada de entrada e nela é preciso especificar o número de
classes e o número de atributos de cada classe:

O diagrama a seguir mostra a aplicação do Embedding.

<img src = '../figures/Embedding_1.png',width=700pt></img>

## Criação da mesma rede, porém agora mais eficiente, com o uso do Embedding

In [11]:
emb = nn.Embedding(n_classes, n_neuronios)
emb

Embedding(20, 5)

In [12]:
my_weights = OrderedDict([
    ('weight',  torch.FloatTensor(W.T.astype(np.float)))])
emb.load_state_dict(my_weights) # inicializa pesos da camada linear
emb.state_dict()

OrderedDict([('weight', 
                  0     0     0     0     0
                  1     2     3     4     5
                  2     4     6     8    10
                  3     6     9    12    15
                  4     8    12    16    20
                  5    10    15    20    25
                  6    12    18    24    30
                  7    14    21    28    35
                  8    16    24    32    40
                  9    18    27    36    45
                 10    20    30    40    50
                 11    22    33    44    55
                 12    24    36    48    60
                 13    26    39    52    65
                 14    28    42    56    70
                 15    30    45    60    75
                 16    32    48    64    80
                 17    34    51    68    85
                 18    36    54    72    90
                 19    38    57    76    95
              [torch.FloatTensor of size 20x5])])

## Predição com mesma sequência: [19, 10, 0, 1, 7, 5, 0, 1, 15, 2]

Confirmamos aqui que a camada Embedding é equivalente à rede densa da entrada categórica feita anteriormente.

In [13]:
data_emb_var = Variable(torch.LongTensor(data.astype(np.int)))
p = emb(data_emb_var)  # predição da rede
p

Variable containing:
   19    38    57    76    95
   10    20    30    40    50
    0     0     0     0     0
    1     2     3     4     5
    7    14    21    28    35
    5    10    15    20    25
    0     0     0     0     0
    1     2     3     4     5
   15    30    45    60    75
    2     4     6     8    10
[torch.FloatTensor of size 10x5]

## Embedding como atributos latentes de um objeto categórico

Podemos interpretar o embedding como uma codificação de atributos latentes de um objeto
categórico. Por exemplo, se estamos codificando filmes, as 5 categorias visto no exemplo
acima poderiam representar a quantidade de suspense, romantismo, aventura, infantil e terror
que um filme possui. Se fosse processar palavras, os atributos poderiam representar o seu
significado (*word embedding*).

O embedding pode ser fixo (não deve ser treinado), quando sabemos exatamente os atributos
das classes ou treináveis, quando queremos que a rede utilize estes atributos como parâmetros
a serem minimizados.

# Aprendizados com este notebook

## Código para jupyter notebook utilizar toda a largura da tela

In [14]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))